In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib widget

In [2]:
import torch
import numpy as np 
import matplotlib.pyplot as plt
import copy

from tqdm import tqdm
from models import FFGC
from plotting_functions import *

In [3]:
model = FFGC()

In [4]:
model_names = model.get_model_list()
model = model.load(model_names[-1]) # load the last model
layer = -1 # valid values are (-1, 1, 3, full)
output_unit = 'automatic'
if output_unit == 'automatic':
    ratemaps, sort_idxs_next = model.forward_ratemaps(layer=layer+2, output_unit=None, sort_idxs=None, res=32)
    output_unit = sort_idxs_next[0]
cumsum = False
# get the sort_idxs from the last model
ratemaps, sort_idxs = model.forward_ratemaps(layer=layer, output_unit=output_unit, sort_idxs=None, res=32)
# plot the ratemaps
ratemaps = np.cumsum(ratemaps, axis=0) if cumsum else ratemaps
fig, _, _ = multiimshow(ratemaps, figsize=(10,10), cmap='seismic', vrange='symmetric')
# create a folder to save the results
import os
path = f'./results/layer={layer}_unit={output_unit}' + ('_cumsum' if cumsum else '')
if not os.path.exists(path):
    os.makedirs(path)
fig.savefig(path + f'/{model.name()}.png', bbox_inches='tight')
plt.close('all')

import gc
for model_name in tqdm(model_names):
    break
    model = model.load(model_name)
    # skip saving fig if it already exists
    if os.path.exists(path + f'/{model.name()}.png'):
        continue
    ratemaps, _ = model.forward_ratemaps(layer=layer, output_unit=output_unit, sort_idxs=sort_idxs, res=32, verbose=False)
    ratemaps = np.cumsum(ratemaps, axis=0) if cumsum else ratemaps
    fig, _, _ = multiimshow(ratemaps, figsize=(10,10), cmap='seismic', vrange='symmetric')
    fig.savefig(path + f'/{model.name()}.png', bbox_inches='tight')
    plt.close('all')
    gc.collect()

Sequential(
  (0): Linear(in_features=2, out_features=64, bias=True)
  (1): ReLU()
)
Pattern formation of output unit 3 in layer 0


  0%|          | 0/100 [00:00<?, ?it/s]
